# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
    create table employees(
        ID_Usr INT primary key,
        name VARCHAR);
    /*3 example rows
    select * from employees limit 3;
    ID_Usr    name
    1344      George StPierre
    2122      Jon jones
    1265      Anderson Silva
    */
    
    create table salary(
        ID_Usr INT primary key,
        year date,
        salary float,
        foreign key (ID_Usr) references employees(ID_Usr));
    /*3 example rows
    select * from salary limit 3;
    ID_Usr    date       salary
    1344      2025       100000.00
    2122      2023       36000.00
    1265      2024       50000.00   
    */
             
    create table studies(
        ID INT,
        ID_Usr INT,
        educational_level INT,  /* 5=phd, 4=Master, 3=Bachelor */
        Institution VARCHAR,
        years date,
        speciality VARCHAR,
        primary key (ID_study, ID_Usr),
        foreign key(ID_Usr) references employees (ID_Usr));
    /*3 example rows
    select * from studies limit 3;
    ID      ID_Usr    educational_level     Institution      Years     Speciality
    22      1344      5                     MIT              2022      AI
    45      2122      3                     UIC Barcelona    2018      Business
    07      1265      3                     Harvard          2020      Medicine
    */
"""} ]



In [ ]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
Question: How Many employes we have with a level of education higher than a Bachelor's degree?
SELECT COUNT(*) AS total_employees
FROM employees e
INNER JOIN studies st ON e.ID_Usr = st.ID_Usr
WHERE st.educational_level > 3;
Question: Return the names of the three people who have had the lowest salary increase in the last three years.
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.year >= DATE_SUB(CURDATE(), INTERVAL 3 YEAR)
GROUP BY e.name
ORDER BY (MAX(s.salary) - MIN(s.salary)) ASC
LIMIT 3;
"""
})

In [5]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [7]:
#new
context_user = context.copy()
print(return_CCRMSQL("The lowest salary from all employees that have studied in the MIT", context_user))

```sql
SELECT MIN(s.salary) AS lowest_salary
FROM employees e
JOIN studies st ON e.ID_Usr = st.ID_Usr
JOIN salary s ON e.ID_Usr = s.ID_Usr
WHERE st.Institution = 'MIT';
```


In [8]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("The lowest salary from all employees that have studied in the MIT", old_context_user))

This is your SQL:
```sql
SELECT MIN(s.salary) AS lowest_salary
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE st.Institution = 'MIT';
```

This SQL query retrieves the lowest salary from all employees who have studied at MIT. It joins the employees, studies, and salary tables on the employee ID, filters the results to only include employees who studied at MIT, and then selects the minimum salary value.


In [10]:
#new
print(return_CCRMSQL("The employee with the longest name nad a salaty higher than 50000", context_user))

```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_Usr = s.ID_Usr
WHERE LENGTH(e.name) = (SELECT MAX(LENGTH(name)) FROM employees)
AND s.salary > 50000;
```


In [11]:
#old
print(return_CCRMSQL("The employee with the longest name nad a salaty higher than 50000", old_context_user))

This is your SQL:
```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE LENGTH(e.name) = (SELECT MAX(LENGTH(name)) FROM employees) AND s.salary > 50000;
```

This SQL query selects the name and salary of the employee with the longest name and a salary higher than 50000. It joins the "employees" and "salary" tables on the ID_usr column, then filters the results to only include the employee with the longest name and a salary greater than 50000.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [12]:
# Version 1

#new
print(return_CCRMSQL("The study specialty with the highest average salary in the company", context_user))

#old
print(return_CCRMSQL("The study specialty with the highest average salary in the company", old_context_user))

```sql
SELECT st.speciality, AVG(s.salary) AS avg_salary
FROM studies st
JOIN salary s ON st.ID_Usr = s.ID_Usr
GROUP BY st.speciality
ORDER BY avg_salary DESC
LIMIT 1;
```
This is your SQL:
```sql
SELECT s.Speciality, AVG(sa.salary) AS avg_salary
FROM studies s
JOIN employees e ON s.ID_usr = e.ID_usr
JOIN salary sa ON s.ID_usr = sa.ID_usr
GROUP BY s.Speciality
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query joins the tables `studies`, `employees`, and `salary` based on the user ID. It calculates the average salary for each study specialty, orders the results by average salary in descending order, and then selects the study specialty with the highest average salary in the company.


In [13]:
# Version 2

#new
print(return_CCRMSQL("The employee with the highest salary for every year of study finalization", context_user))

#old
print(return_CCRMSQL("The employee with the highest salary for every year of study finalization", old_context_user))

To find the employee with the highest salary for every year of study finalization, you can use the following SQL query:

```sql
WITH max_salary_per_year AS (
    SELECT st.years AS study_year, MAX(s.salary) AS max_salary
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_Usr
    JOIN studies st ON e.ID_Usr = st.ID_Usr
    GROUP BY st.years
)

SELECT e.name AS employee_name, m.study_year, m.max_salary AS highest_salary
FROM employees e
JOIN salary s ON e.ID_Usr = s.ID_Usr
JOIN studies st ON e.ID_Usr = st.ID_Usr
JOIN max_salary_per_year m ON st.years = m.study_year AND s.salary = m.max_salary;
```

In this query:
1. The `max_salary_per_year` CTE calculates the maximum salary for each year of study finalization.
2. The main query then joins the `employees`, `salary`, `studies`, and `max_salary_per_year` CTE to retrieve the employee name, study year, and highest salary for that year.

This query will return the employee with the highest salary for every year of study finalization.
T

In [14]:
# Version 3

#new
print(return_CCRMSQL("The *percentage* increase in workforce per year with respect to the previous year", context_user))

#old
print(return_CCRMSQL("The *percentage* increase in workforce per year with respect to the previous year", old_context_user))

To calculate the percentage increase in the workforce per year with respect to the previous year, you can use the following SQL query:

```sql
WITH EmployeeCount AS (
    SELECT 
        year,
        COUNT(*) AS total_employees
    FROM salary
    GROUP BY year
)
SELECT 
    e1.year,
    e1.total_employees AS current_year_employees,
    e2.total_employees AS previous_year_employees,
    ((e1.total_employees - e2.total_employees) * 100.0 / e2.total_employees) AS percentage_increase
FROM EmployeeCount e1
JOIN EmployeeCount e2 ON e1.year = e2.year + 1
ORDER BY e1.year;
```

In this query:
- We first calculate the total number of employees for each year from the `salary` table.
- Then, we join this data with itself to compare the total number of employees in the current year with the total number of employees in the previous year.
- Finally, we calculate the percentage increase in the workforce per year with respect to the previous year.

This query will provide you with the year, the tot

# Conclusions

In this notebook, we explored different prompt engineering strategies for generating SQL queries from natural language questions using large language models. We compared the effectiveness of the "old" prompt (minimal schema, little context) with a "new" prompt (detailed schema, sample data, and few-shot examples), and then experimented with three creative prompt variations.

## Key Findings

- **Few-shot examples improve accuracy:** Including example questions and their corresponding SQL queries in the prompt (few-shot learning) helped the model better understand the expected output format and improved the correctness of the generated SQL.
- **Schema clarity matters:** Providing clear table schemas and sample data rows made it easier for the model to infer relationships between tables and select appropriate columns, reducing hallucinations and errors.
- **Prompt style influences output:** Prompts that encouraged step-by-step reasoning produced more interpretable results, while those that requested only SQL code led to concise but sometimes less accurate answers.
- **Complex queries remain challenging:** For more complex analytical questions (e.g., calculating percentage increases or multi-table aggregations), the model sometimes struggled, especially with minimal context or ambiguous instructions.
- **Natural language descriptions help:** Describing tables and columns in plain English, in addition to SQL schema, made the prompt more accessible and sometimes improved the model's understanding of the data.

## Lessons Learned

- The quality and structure of the prompt have a significant impact on the performance of LLMs for text-to-SQL tasks.
- Providing both schema and example data, along with a few illustrative queries, is a best practice for maximizing accuracy.
- For production use, prompts should be tailored to the complexity of the queries expected and the familiarity of the model with the database schema.
- Iterative prompt design and testing are essential to identify and mitigate cases where the model may hallucinate or generate invalid SQL.

Overall, prompt engineering is a powerful tool for guiding LLMs in structured data tasks, and thoughtful design can greatly enhance reliability and usefulness in real-world applications.